In [140]:
# coding: utf-8
import math
import itertools
import statistics
from enum import Enum
import zipfile

import numpy as np
from scipy import stats
from scipy.stats import norm
from statsmodels.stats.power import TTestIndPower


# Configuration
P_VALUE_THRESHOLD = 0.05
TRT_EFFECT_THRESHOLD = -0.6
NUMBER_THRESHOLD = (60, 180)
POWER_THRESHOLD = 0.8


def get_quartiles(ary):
    unit = 100/12
    return [i*unit for i in range(1, 12)]

# Generate all possible rules
def get_rule_1(patients):
    
    
    discrete_values = [(0,), (1,), (2,), (0, 1), (1, 2)]
    for param in range(1, 21):
        for dv in discrete_values:
            yield (param, dv, 'equal')

    for param in range(21, 41):
        sorted_df = patients[patients['trt'] == 1].sort_values('x{}'.format(param))
        slope, _, _, _, _ = stats.linregress(sorted_df['x{}'.format(param)], sorted_df['y'])
        
        continuous_values = get_quartiles(patients['x{}'.format(param)])
        for cv in continuous_values:
            if slope < 0:
                yield (param, cv, 'larger')
            elif slope > 0:
                yield (param, cv, 'smaller')
    
    #yield ((4, (1, 2), 'equal'), (22, 57.7, 'larger'), 'and')
    #yield (22, 60 , 'smaller')
    

def get_rule_2(patients):
    rules = [r for r in get_rule_1(patients)]
    for i in range(0, len(rules)):
        for j in range(i, len(rules)):
            if rules[i][0] == rules[j][0]:
                continue

            yield (rules[i], rules[j], 'and')
            yield (rules[i], rules[j], 'or')

def get_cartesian_product(did):
    df = pandas.read_pickle('rules/{}_1.pickle'.format(did))
    single_rules = df['rule']

    for i in range(0, len(single_rules)):
        k = i+1
        while k < len(single_rules):
            yield (single_rules[i], single_rules[k], 'and')
            yield (single_rules[i], single_rules[k], 'or')
            k = k+1
    
def check_group_2(patient, rule):
    if (rule[2] == 'or') | (rule[2] == 'and'):
        rule_1, rule_2, flag = rule
        if flag == 'and':
            return (check_group_1(patient, rule_1) & check_group_1(patient, rule_2))
            
        elif flag == 'or':
             return (check_group_1(patient, rule_1) | check_group_1(patient, rule_2))
    else:
        return check_group_1(patient, rule)
    
def check_group_1(patient, rule):
    idx, val, flag = rule
    
    if flag == 'equal':
        if len(val) > 1:
            return ((patient['x{}'.format(idx)] == val[0])|(patient['x{}'.format(idx)] == val[1]))
        else:
            return patient['x{}'.format(idx)] == val
        
    elif flag == 'smaller':
        return patient['x{}'.format(idx)] <= val
  
    elif flag == 'larger':
         return patient['x{}'.format(idx)] > val

def evaluation(d):

    # Subgroup
    gp = d[d['group'] == 1]
    gp_1 = gp[gp['trt'] == 1]
    gp_0 = gp[gp['trt'] == 0]

    gp_1_num = len(gp_1)
    gp_0_num = len(gp_0)
    gp_num = gp_1_num + gp_0_num
    if gp_1_num < 2 or gp_0_num < 2:
        return None

    gp_trt_effect = statistics.mean(gp_1['y']) - statistics.mean(gp_0['y'])

    gp_1_stdev = statistics.stdev(gp_1['y'])
    gp_0_stdev = statistics.stdev(gp_0['y'])
    tmp_1 = (gp_1_num-1)*math.pow(gp_1_stdev, 2)
    tmp_0 = (gp_0_num-1)*math.pow(gp_0_stdev, 2)
    gp_stdev = math.sqrt((tmp_1 + tmp_0)/(gp_1_num + gp_0_num - 2))

    gp_ratio = gp_0_num / gp_1_num
    _, gp_p = stats.ranksums(gp_1['y'], gp_0['y'])

    gp_effect_size = gp_trt_effect/gp_stdev
    
    tmp_1 = math.pow(gp_stdev, 2) / math.pow(gp_num, 0.5)
    gp_power = norm.cdf((-gp_trt_effect/tmp_1)-1.96)

    # Nongroup
    gn = d[d['group'] == 0]
    gn_1 = gn[gn['trt'] == 1]
    gn_0 = gn[gn['trt'] == 0]

    gn_1_num = len(gn_1)
    gn_0_num = len(gn_0)
    gn_num = gn_1_num + gn_0_num
    if gn_1_num < 2 or gn_0_num < 2:
        return None

    gn_trt_effect = statistics.mean(gn_1['y']) - statistics.mean(gn_0['y'])

    gn_1_stdev = statistics.stdev(gn_1['y'])
    gn_0_stdev = statistics.stdev(gn_0['y'])
    tmp_1 = (gn_1_num-1)*math.pow(gn_1_stdev, 2)
    tmp_0 = (gn_0_num-1)*math.pow(gn_0_stdev, 2)
    gn_stdev = math.sqrt((tmp_1 + tmp_0)/(gn_1_num + gn_0_num - 2))

    gn_effect_size = gn_trt_effect/gn_stdev

    _, gn_p = stats.ranksums(gn_1['y'], gn_0['y'])

    # Inter
    trt_effect_1 = statistics.mean(gp_1['y']) - statistics.mean(gn_1['y'])
    trt_effect_0 = statistics.mean(gp_0['y']) - statistics.mean(gn_0['y'])

    _, p_1 = stats.ranksums(gp_1['y'], gn_1['y'])
    _, p_0 = stats.ranksums(gp_0['y'], gn_0['y'])
    
    
    tmp_1 = (gp_1_num-1)*math.pow(gp_1_stdev, 2)
    tmp_0 = (gn_1_num-1)*math.pow(gn_1_stdev, 2)
    stdev_1 = math.sqrt((tmp_1 + tmp_0)/(gp_1_num + gn_1_num - 2))
    
    tmp_1 = math.pow(stdev_1, 2) / math.pow(gp_1_num+gn_1_num, 0.5)
    power_1 = norm.cdf((-trt_effect_1/tmp_1)-1.96)
    

    '''
    # Filter
    if gp_trt_effect > TRT_EFFECT_THRESHOLD:
        return None
    #elif trt_effect_1 > TRT_EFFECT_THRESHOLD:
    #    return None
    elif gp_p > P_VALUE_THRESHOLD:
        return None
    #elif p_1 > P_VALUE_THRESHOLD:
    #    return None
    elif gp_power < POWER_THRESHOLD:
        return None
    elif power_1 < POWER_THRESHOLD:
        return None
    #elif gp_num < NUMBER_THRESHOLD[0] or gp_num > NUMBER_THRESHOLD[1]:
    #    return None
    '''
    
    gp_1_mean = statistics.mean(gp_1['y'])
    gp_0_mean = statistics.mean(gp_0['y'])
    gn_1_mean = statistics.mean(gn_1['y'])
    gn_0_mean = statistics.mean(gn_0['y'])
    return gp_effect_size, gp_num, gp_trt_effect, gp_p, gp_1_mean, gp_1_num, gp_0_mean, gn_trt_effect, gn_p, gn_1_mean, gn_1_num, gn_0_mean,trt_effect_1, p_1, trt_effect_0, p_0, gp_power, power_1

def find_good_rules(t):
    # Load parameters
    did, d, rule_len = t

    rules = []
    
    # Get all possible rules
    if rule_len == 1:
        possible_rules = [r for r in get_rule_1(d)]
    elif rule_len == 1.5:
        possible_rules = [r for r in get_cartesian_product(did)]
    elif rule_len == 2:
        possible_rules = [r for r in get_rule_2(d)]
    else:
        print('wrong parameter.')
        return
    
    for rule in possible_rules:
        d['group'] = 0
        labels = check_group_2(d, rule)
        d.loc[labels, 'group'] = 1

        # Evaluate the treatment effect according each rule
        parts = evaluation(d)
        if parts is None:
            continue

        gp_effect_size, gp_num, gp_trt_effect, gp_p, gp_1_mean, gp_1_num, gp_0_mean, gn_trt_effect, gn_p, gn_1_mean, gn_1_num, gn_0_mean,trt_effect_1, p_1, trt_effect_0, p_0, gp_power, power_1 = parts
        rules.append((rule, gp_effect_size, gp_num, gp_trt_effect, gp_p, gp_1_mean, gp_1_num, gp_0_mean, gn_trt_effect, gn_p, gn_1_mean, gn_1_num, gn_0_mean,trt_effect_1, p_1, trt_effect_0, p_0, gp_power, power_1))

    # Save to piackle
    import pandas
    rules = pandas.DataFrame(rules, columns=['rule', 'gp_effect_size', 'gp_num', 'gp_trt_effect', 'gp_p', 'gp_1_mean', 'gp_1_num', 'gp_0_mean', 'gn_trt_effect', 'gn_p', 'gn_1_mean', 'gn_1_num', 'gn_0_mean', 'trt_effect_1', 'p_1', 'trt_effect_0', 'p_0', 'gp_power', 'power_1'])
    rules.to_pickle('rules/{}_{}.pickle'.format(did, rule_len))
    rules.to_csv('rules/{}_{}.csv'.format(did, rule_len))
    return rules

In [143]:
did = 2
d = training_data[training_data['dataset'] == did].copy()

start_time = time.time()
result = find_good_rules((did, d, 1))
print("\n--- {} seconds ---\n".format(time.time() - start_time))

# result.to_csv('dataset{}.csv'.format(did))


--- 4.165822982788086 seconds ---



In [142]:
result

,rule,gp_effect_size,gp_num,gp_trt_effect,gp_p,gp_1_mean,gp_1_num,gp_0_mean,gn_trt_effect,gn_p,gn_1_mean,gn_1_num,gn_0_mean,trt_effect_1,p_1,trt_effect_0,p_0,gp_power,power_1
0,"(1, (0,), equal)",-0.101172,112,-0.137755,0.506411,7.189333,78,7.327088,-0.738397,0.003649,6.906759,83,7.645156,0.282574,0.120519,-0.318067,0.575866,0.120269,7.666127e-06
1,"(1, (1,), equal)",-0.522146,105,-0.642529,0.024021,6.861357,70,7.503886,-0.327860,0.150203,7.183890,91,7.511750,-0.322533,0.079640,-0.007864,0.711310,0.991529,7.728592e-01
2,"(1, (2,), equal)",-0.905929,23,-0.988369,0.054538,7.151231,13,8.139600,-0.382559,0.041504,7.034209,148,7.416768,0.117021,0.732903,0.722832,0.165761,0.978427,1.712780e-03
3,"(1, (0, 1), equal)",-0.293848,217,-0.382559,0.041504,7.034209,148,7.416768,-0.988369,0.054538,7.151231,13,8.139600,-0.117021,0.732903,-0.722832,0.165761,0.913855,1.602870e-01
4,"(1, (1, 2), equal)",-0.609333,128,-0.738397,0.003649,6.906759,83,7.645156,-0.137755,0.506411,7.189333,78,7.327088,-0.282574,0.120519,0.318067,0.575866,0.999904,6.568422e-01
5,"(2, (0,), equal)",-0.197044,117,-0.267725,0.213829,7.164358,81,7.432083,-0.650597,0.015580,6.921450,80,7.572047,0.242908,0.180061,-0.139963,0.972516,0.347774,3.373797e-05
6,"(2, (1,), equal)",-0.387203,102,-0.460864,0.120135,7.031507,67,7.492371,-0.468590,0.046376,7.052319,94,7.520909,-0.020812,0.832954,-0.028538,0.748399,0.907501,3.687377e-02
7,"(2, (2,), equal)",-1.221997,21,-1.566394,0.020479,6.354231,13,7.920625,-0.357587,0.047461,7.104216,148,7.461803,-0.749985,0.037652,0.458822,0.558531,0.991994,9.999948e-01
8,"(2, (0, 1), equal)",-0.279701,219,-0.357587,0.047461,7.104216,148,7.461803,-1.566394,0.020479,6.354231,13,7.920625,0.749985,0.037652,-0.458822,0.558531,0.899311,4.158095e-17
9,"(2, (1, 2), equal)",-0.534796,123,-0.650597,0.015580,6.921450,80,7.572047,-0.267725,0.213829,7.164358,81,7.432083,-0.242908,0.180061,0.139963,0.972516,0.998224,5.259189e-01


In [127]:
import time

import pandas
from ipyparallel import Client

import subgroup_analysis.brute_force as b
import subgroup_analysis.preprocessing as preprocessing


def brute_force(df, start_dataset, end_dataset, rule_len):
    
    # Engine initialization
    c = Client()
    v = c[:]
    v.block = True
    
    # Run brute-force parallelly
    # Parameters: Dataset ID, Dataframe, Rule length
    # Return: Good Rules
    params = []
    for did in range(start_dataset, end_dataset+1):
        d = df[df['dataset'] == did].copy()
        params.append((did, d, rule_len))
    
    return v.map(preprocessing.find_good_rules, params)

In [130]:
training_data = pandas.read_csv('datasets/Training_Data.csv')
testing_data = pandas.read_csv('datasets/Data.csv')

In [5]:
# Save all rules
start_time = time.time()
results = brute_force(training_data, 2, 2, 1)
print("\n--- {} seconds ---\n".format(time.time() - start_time))


--- 7.527747869491577 seconds ---



In [129]:
import time

import pandas
import subgroup_analysis.preprocessing as preprocessing


def check_group(patient, rule):
    idx, val, flag = rule

    achieve_num = 0
    if flag == 'equal':
        if patient['x{}'.format(idx)] in val:
            achieve_num += 1

    elif flag == 'smaller':
        if patient['x{}'.format(idx)] <= val:
            achieve_num += 1

    elif flag == 'larger':
        if patient['x{}'.format(idx)] > val:
            achieve_num += 1

    if achieve_num == 1:
        return 1

    return 0

def generate_result(df, start, end, rule_2=False):
    # Set default group number to every patients
    if 'group' not in df:
        df['group'] = 0
    
    result = []
    no_rules = 0
    for did in range(start, end+1):
        rules = pandas.read_pickle('rules/{}_1.pickle'.format(did))
        if rule_2:
            rules = rules.append(pandas.read_pickle('rules/{}_2.pickle'.format(did)))
        
        if len(rules.index) == 0:
            non_rules += 1
            result.append(None)
            continue

        gp_effect_size_rank = []
        gp_trt_effect_rank = []
        #trt_effect_1_rank = []
        
        for row in rules.iterrows():
            rule = row[1]['rule']
            gp_effect_size = row[1]['gp_effect_size']
            gp_trt_effect = row[1]['gp_trt_effect']
            #trt_effect_1 = row[1]['trt_effect_1']
            
            gp_effect_size_rank.append((rule, gp_effect_size))
            gp_trt_effect_rank.append((rule, gp_trt_effect))
            #trt_effect_1_rank.append((rule, trt_effect_1))
    
        # Ranking
        gp_effect_size_rank = sorted(gp_effect_size_rank, key=lambda x: x[1])
        gp_trt_effect_rank = sorted(gp_trt_effect_rank, key=lambda x: x[1])
        #trt_effect_1_rank = sorted(trt_effect_1_rank, key=lambda x: x[1])

        rank_dict = dict()
        for i in range(0, 10):
            try:
                item = gp_effect_size_rank[i]
                if item[0] in rank_dict:
                    rank_dict[item[0]] += 10-i
                else:
                    rank_dict[item[0]] = 10-i
            except:
                pass

            try:
                item = gp_trt_effect_rank[i]
                if item[0] in rank_dict:
                    rank_dict[item[0]] += 10-i
                else:
                    rank_dict[item[0]] = 10-i
            except:
                pass

            '''
            try:
                item = trt_effect_1_rank[i]
                if item[0] in rank_dict:
                    rank_dict[item[0]] += 10-i
                else:
                    rank_dict[item[0]] = 10-i
            except:
                pass
            '''

        rank = []
        for k, v in rank_dict.items():
            rank.append((k, v))
        rank = sorted(rank, reverse=True, key=lambda x: x[1])
        
        # Grouping patients using the rule with the highest score
        rule = rank[0][0]
        result.append(rule)
    
    print('Databases with no rules: {}'.format(no_rules))
    return result

# Merge the results
def merge_results(df, rules):
    result = pandas.DataFrame([], columns=(['id'] + ['dataset_{}'.format(i) for i in range(1, 1201)]))
    result['id'] = range(1, 241)

    for did in range(1, 1201):
        d = df[df['dataset'] == did].copy()

        # (16, (1,), 'equal')
        
        for pid in range(1, 241):
            patient = d[d['id'] == pid].iloc[0]
            g = check_group(patient, rules[did-1]) if rules[did-1] is not None else 0
            result.loc[pid-1, 'dataset_{}'.format(did)] = g

    return result

#for i in range(1, 241):
#            g = check_group(df.loc[(df['id'] == i) & (df['dataset'] == did)], rule)
#            df.loc[(df['id'] == i) & (df['dataset'] == did), 'group'] = g

In [133]:
start_time = time.time()
#rules = generate_result(training_data, 2, 2, rule_2=False)
#result = merge_results(testing_data, rules)
print("\n--- {} seconds ---\n".format(time.time() - start_time))


--- 7.915496826171875e-05 seconds ---



In [131]:
rules

[((32, 33.333333333333336, 'smaller'),
  (40, 16.666666666666668, 'smaller'),
  'and')]

In [64]:
# Save rules
did = 3
d = training_data[training_data['dataset'] == did].copy()

start_time = time.time()
# result = find_good_rules((did, d, 3))
print("\n--- {} seconds ---\n".format(time.time() - start_time))


--- 0.00010514259338378906 seconds ---



In [3]:
did = 4
d = training_data[training_data['dataset'] == did].copy()

start_time = time.time()
result = b.find_good_rules((did, d, 1))
print("\n--- {} seconds ---\n".format(time.time() - start_time))

# result.to_csv('dataset{}.csv'.format(did))

Dataset 4: (30, 72.675000000000011, <Comparison.larger: 1>) (29)

--- 141.41580390930176 seconds ---



In [3]:
# ipcluster start -n 4

# Merge the results
def merge_results(df, rules):
    result = pandas.DataFrame([], columns=(['id'] + ['dataset_{}'.format(i) for i in range(1, 1201)]))
    result['id'] = range(1, 241)

    for did in range(1, 1201):
        d = df[df['dataset'] == did].copy()
        # print('dataset {}'.format(did))

        for pid in range(1, 241):
            patient = d[d['id'] == pid].iloc[0]
            g = b.check_group(patient, rules[did-1].iloc[0][0]) if rules[did-1] is not None else 0
            result.loc[pid-1, 'dataset_{}'.format(did)] = g

    return result

start_time = time.time()
# result = merge_results(testing_data, results)
# result.to_csv('result.csv')
print("\n--- {} seconds ---\n".format(time.time() - start_time))



--- 0.00014400482177734375 seconds ---



In [334]:
# Merge the results
def merge_results(results):    
    final_result = results[0][0]['group'].copy()
    for i in range(8):
        for j in range(288000):
            if results[i][0]['group'][j] == 1:
                final_result[j] = 1
    return final_result

# Export the result as csv
def save_to_csv(result):
    df = pandas.DataFrame([], columns=(['id'] + ['dataset_{}'.format(i) for i in range(1, 1201)]))
    c = 0
    for idx in range(1, 241):
        df.loc[idx-1, 'id'] = idx
        
        print(idx)

        for did in range(1, 1201):
            df.loc[idx-1, 'dataset_{}'.format(did)] = result[c]
            c += 1
    
    df.to_csv('result_bf_1.csv')

# save_to_csv(merge_results(results))
# save_to_csv(final_result)

In [193]:
def generate_testing_result(did, rule):
    # df = pandas.read_csv('datasets/InnoCentive_9933623_Data.csv')
    df = pandas.read_csv('datasets/InnoCentive_9933623_Training_Data.csv')
    df = df[df['dataset'] == did]
    df['group'] = 0
    
    for i in range(1, 241):
        patient = df[(df['dataset'] == did) & (df['id'] == i)].iloc[0]

        if rule is None:
            g = 0
        else:
            g = check_group(patient, rule)

        df.loc[(df['dataset'] == did) & (df['id'] == i), 'group'] = g
    
    return df

def save_testing_csv(result, dataset, name='default'):
    df = pandas.DataFrame([], columns=(['id'] + ['dataset_{}'.format(i) for i in range(1, 1201)]))
    df['id'] = range(1, 241)
    
    for did in range(1, 1201):
        df['dataset_{}'.format(did)] = 0
        if did == dataset:
            for pid in result['id']:
                df.loc[pid-1, 'dataset_{}'.format(did)] = 1
    
    df.to_csv('result_bf_{}.csv'.format(name))
    
    zf = zipfile.ZipFile('result_bf_{}.zip'.format(name), 'w', zipfile.ZIP_DEFLATED)
    zf.write('result_bf_{}.csv'.format(name))


In [290]:
did = 2
r = generate_testing_result(did, (29, 51.3, Comparison.smaller))
# save_testing_csv(r, did, name='x18=0')

In [291]:

# Evaluate the treatment effect according each rule
parts = evaluation(r)

print('effect_size, non_effect_size, trt_effect, nongroup_mean, p_value,    non_p_value, number, power')
print('{:.8f}, {:.12f}, {:.7f}, {:.10f}, {:.8f}, {:.10f}, {}, {:.8f}'.format(*parts))

effect_size, non_effect_size, trt_effect, nongroup_mean, p_value,    non_p_value, number, power
-0.47160753, 0.000795688370, -0.6529163, 0.0009611111, 0.00740080, 0.4984384793, 129, 1.00000000
